In [1]:
import tensorflow as tf
import pandas as pd
import datetime
import numpy as np
import os
from tensorflow.python import pywrap_tensorflow

## Preprocessing

In [2]:
'''
PARAMETERS TO CHANGE
'''
stock_symbol = 'bac'
train_model = True
test_model = True
''''''

training_data_dir = './training_data/'
y_columns = ['ppo', 'rsi']
#assert(train_model ^ test_model)
model_dir = "./regularizer_models/" + stock_symbol + '/'

if train_model:
    training_data = pd.read_csv(training_data_dir + stock_symbol + '_training_data.csv')
    unneeded_columns = ['window']

    x_columns = list(training_data.columns)
    for to_remove in (y_columns + unneeded_columns):
        x_columns.remove(to_remove)

    train_x = training_data[x_columns].values
    train_y = training_data[y_columns].values

    model_dir = "./regularizer_models/" + stock_symbol + '/'

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
        
if test_model:
    test_data = pd.read_csv(training_data_dir + stock_symbol + '_test_data.csv')
    
    unneeded_columns = ['Date_to_use_on']
    x_columns = list(test_data.columns)

    for to_remove in (unneeded_columns):
        x_columns.remove(to_remove)

    
    test_x = test_data[x_columns].values
    
    
    

## ANN Model

In [3]:
# Parameters
learning_rate = 0.1
num_epochs = 30000
batch_size = 25
display_epoch = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = len(x_columns) # Number of features
num_output = len(y_columns) # PPO, RSI parameters


def ANN(X):
    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
        'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
        'out': tf.Variable(tf.random_normal([n_hidden_2, num_output]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden_1])),
        'b2': tf.Variable(tf.random_normal([n_hidden_2])),
        'out': tf.Variable(tf.random_normal([num_output]))
    }

    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(X, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)    
    
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    
    # Output fully connected layer with a neuron for each output
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    out = tf.square(out_layer)
    
    return out

X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_output])

output = ANN(X)

# Loss & Optimizer
loss_op = tf.losses.mean_squared_error(Y, output, weights=1)
train_op = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_op)
# Accuracy
accuracy =  tf.abs(tf.reduce_mean(tf.round(output) - Y))

In [4]:
if train_model:

    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        # Run initializer
        sess.run(init)
        for epoch in range(num_epochs):
            
            shuffled_indexes = np.random.permutation(len(train_x))
    
            
            for i in shuffled_indexes[::batch_size]:
                batch_x = train_x[i:i+batch_size]
                batch_y = train_y[i:i+batch_size]

                sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})

            if epoch % display_epoch == 0 or epoch == 1:
                loss, acc = sess.run([loss_op, accuracy], feed_dict={X: train_x, Y: train_y})

                print("Epoch " + str(epoch) + ", Loss= " + \
                      "{:.4f}".format(loss) + ", Training Error= " + \
                      "{:.3f}".format(acc))

            if acc <= 0.6 and loss < 20:
                break
        print(sess.run(output, feed_dict={X: train_x}))
        saver.save(sess, model_dir + stock_symbol + '_regularizer.ckpt')
        
        
        print('Model Saved')

      

Epoch 0, Loss= 10397154.0000, Training Error= 362.263
Epoch 1, Loss= 13488405.0000, Training Error= 345.040
Epoch 100, Loss= 915.7308, Training Error= 9.188
Epoch 200, Loss= 28.0386, Training Error= 0.133
Epoch 300, Loss= 16.3076, Training Error= 0.140
[[2.471811  4.8930407]
 [2.7852898 4.967439 ]
 [1.8384863 4.728694 ]
 ...
 [2.7852898 4.967439 ]
 [2.0190012 4.777795 ]
 [2.7235622 4.9530907]]
Model Saved


In [9]:
if test_model:
    
    saver = tf.train.Saver()
    with tf.Session() as sess:
        '''var_name_list = [v.name for v in tf.trainable_variables()]
        print(var_name_list)
        reader = pywrap_tensorflow.NewCheckpointReader(model_dir + stock_symbol + '_regularizer.ckpt')
        var_to_shape_map = reader.get_variable_to_shape_map()'''
        
        saver.restore(sess, model_dir + stock_symbol + '_regularizer.ckpt')
        print("Model restored.")
        predicted = tf.round(sess.run(output, feed_dict={X : test_x})).eval()
        predicted = np.array(predicted)
        
    result = pd.DataFrame(predicted, columns = y_columns)
    result['Date_to_use_on'] = test_data['Date_to_use_on']
    result = result[['Date_to_use_on'] + y_columns]
    result.to_csv(training_data_dir+ stock_symbol + '_test_parameters.csv', index=False)
    

INFO:tensorflow:Restoring parameters from ./regularizer_models/bac/bac_regularizer.ckpt
Model restored.


In [10]:
result

,Date_to_use_on,ppo,rsi
0,2009-11-20,2.0,5.0
1,2009-11-23,3.0,5.0
2,2009-11-24,2.0,5.0
3,2009-11-25,2.0,5.0
4,2009-11-27,3.0,5.0
5,2009-11-30,2.0,5.0
6,2009-12-01,3.0,5.0
7,2009-12-02,2.0,5.0
8,2009-12-03,3.0,5.0
9,2009-12-04,3.0,5.0


# 